In [1]:
import numpy as np
import time
import scipy.integrate as integral
import pandas as pd
import matplotlib.pyplot as plt

import netgen.meshing
from ngsolve import *
from netgen.occ import * 
from ngsolve.webgui import Draw


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
def mesh_uniform_interval(a,b,N):
    netgen_mesh = netgen.meshing.Mesh(dim=1)
    delta_space = (b-a)/N

    pnums = []
    for i in range(N+1):
        pnums.append(netgen_mesh.Add( netgen.meshing.MeshPoint( netgen.meshing.Pnt(a+i*delta_space,0,0 ))))
    idx = netgen_mesh.AddRegion("material", dim=1)
    for i in range(0,N):
        netgen_mesh.Add( netgen.meshing.Element1D ([pnums[i], pnums[i+1]], index=idx) )
    idx_left = netgen_mesh.AddRegion("left", dim=0)
    idx_right = netgen_mesh.AddRegion("right", dim=0)
    netgen_mesh.Add( netgen.meshing.Element0D(pnums[0], index=idx_left) )
    netgen_mesh.Add( netgen.meshing.Element0D(pnums[-1], index=idx_right) ) 
    
    mesh = Mesh(netgen_mesh)
    return mesh

In [5]:
mesh = mesh_uniform_interval(0, 1, 10)
Draw(mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [6]:
print(mesh.GetBoundaries())

('left', 'right')


In [ ]:
# parameters
nu0, c0 = 6e6, 1540
tau= 1/nu0; mean_Gaussian= 2*tau 
chi= lambda u: 1/tau*exp(-0.5*((u-mean_Gaussian)/tau)*((u-mean_Gaussian)/tau))
#chi= lambda u: exp(-u*u)
f = lambda t: (t>0)*chi(t) #lo dej+e sin límite superior

def A(t_end):
    A= (t_end>0)*integral.quad(lambda t: chi(t)/c0, 0, t_end)[0]
    #A= integral.quad(lambda t: f(t)/c0, 0, t_end)[0]
    return A

t=np.arange(-4*tau, 7*tau, tau/10)
plt.plot(t, f(t))
plt.show()

dt = tau/10; 
# k_max=2300
t0=0*tau
t_final = 26*tau
k_max= round((t_final - t0)/dt)
print(f'Calcula {k_max} pasos de tiempo')



# inicializar con cualquier valor donde f no se haga cero, para no crear una ZeroCoefficientFunction
t_initialization_value = mean_Gaussian+tau 

In [ ]:
def wave_solver(mesh, tend):
    '''
    M y' = A y + F(t)
    '''
    t_ngsolve = Parameter(0)    
    f = sin(pi*(x-t))


    U = H1(mesh, order=3, dirichlet="left|right")
    V = H1(mesh, order=3, dirichlet="left|right")
    fes = U*V
    uh, vh = fes.TrialFunction()
    du, dv = fes.TestFunction()
    a = BilinearForm(fes, symmetric=True)
    a += (vh*du + grad(uh)*grad(dv))*dx
    a.Assemble()

    m = BilinearForm(fes, symmetric=True)
    m += (uh*du + vh*dv)*dx
    m.Assemble()    

    mstar = m.mat.CreateMatrix()
    mstar.AsVector().data = m.mat.AsVector() +  (dt) * a.mat.AsVector()
    invmstar = mstar.Inverse(freedofs=fes.FreeDofs())

    F = LinearForm(fes)
    F = f*dv*dx

    # Initial condition
    gfU = GridFunction(fes)
    gfu, gfv = gfU.components
    gfu.Set(t_initialization_value, definedon=mesh.Boundaries("left"))

    t_ngsolve = Parameter(0)    
    t = t0
    while t< tend - 0.5 * dt:
        t_ngsolve.Set(t+dt)
        F.Assemble()
        
        gfD = GridFunction(fes)
        gfD.components[0].Set(uD.vec, BND)
        gfD.components[1].Set(vD.vec, BND)

        res = dt * (F.vec) + m.mat * gfU.vec - mstar * gfD.vec
        gfU.vec.data = gfD.vec + invmstar * res

    return gfU

In [7]:




# es importante pasar el t al timeparams (de la class ngsolve.Parameter), no el t0 (un float),
# para que el pulso vaya actualizando su valor a medida que el tiempo t avanza con los t+=dt/2
t = Parameter(t_initialization_value)  
f=CF(f(t))

# Elegir el valor inicial de t 
t0=0; t.Set(t0)

ui=CF(0)
vi=CF(0)

# (gf, err, max_err, err_rel, max_err_rel, err_rel_prom) = WaveSolver(mesh, [ui, vi], f, timeparams=[dt, k_max, t], theta=theta, radio=chamber_radio)

W1 = H1(mesh, order=1, dirichlet = "left|right")#, complex=True)
W2 = H1(mesh, order=1)#, complex=True)
fes = FESpace([W1, W2])
uh, vh = fes.TrialFunction()    
w1, w2 = fes.TestFunction()     #Test function

c = CoefficientFunction(c0)

gf = GridFunction(fes)      #future solution
gfu, gfv = gf.components

a = BilinearForm(fes)
a += uh*w1*dx - dt*vh*w1*dx 
a += vh*w2*dx + dt*c**2*grad(uh)*grad(w2)*dx
a.Assemble()
ainv = a.mat.Inverse(freedofs=fes.FreeDofs())

L = LinearForm(fes)
L += gfu*w1*dx + gfv*w2*dx
L += dt*f*w2*ds("gamma")

gfu.Set(ui)
gfv.Set(vi)

for k in range(1, k_max):
    t += dt/2
    #Draw(CF(1), mesh, "f", eval_function="gamma", min=-1, max=1)#, settings={"camera": {"transformations": [{"type": "rotateY", "angle": 45}]}})
    L.Assemble()

    # Actualización del vector solución de NGSolve
    gf.vec.data = ainv*L.vec

    # Actualización tiempo
    t += dt/2

    print(Integrate (gfu*Conj(gfu), mesh))
    

# elapsed_time[s] = time.time() - tiempo
# print('tiempo de computo para ', int(0.06/(sr))*10, 'nodos en y:', elapsed_time)
elapsed_time = time.time() - tiempo
print(f'tiempo de computo: {elapsed_time}')

TypeError: __call__(): incompatible function arguments. The following argument types are supported:
    1. (self: ngsolve.fem.CoefficientFunction, mip: ngsolve.fem.BaseMappedIntegrationPoint) -> object
    2. (self: ngsolve.fem.CoefficientFunction, x: float, y: Optional[float] = None, z: Optional[float] = None) -> ngcomp::PointEvaluationFunctional
    3. (self: ngsolve.fem.CoefficientFunction, arg0: ngsolve.fem.CoordinateTrafo) -> ngsolve.fem.CoefficientFunction
    4. (self: ngsolve.fem.CoefficientFunction, arg0: ngsolve.fem.MeshPoint) -> object
    5. (self: ngsolve.fem.CoefficientFunction, arg0: numpy.ndarray[ngsolve.fem.MeshPoint]) -> numpy.ndarray

Invoked with: <ngsolve.fem.CoefficientFunction object at 0x7eafab4b2fc0>, <ngsolve.fem.Parameter object at 0x7eafab540090>